<a href="https://colab.research.google.com/github/divya2648/CS682/blob/main/CS682_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pycountry

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.1 MB 17.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681845 sha256=e8ad3ad67a68e12faa32db5952d6d11d3007bd6191b4f27132563168789513a2
  Stored in directory: /root/.cache/pip/wheels/0e/06/e8/7ee176e95ea9a8a8c3b3afcb1869f20adbd42413d4611c6eb4
Successfully built pycountry


In [16]:
import pandas as pd
import requests
import json
import csv
from geopy import distance
from geopy.geocoders import Nominatim
from google.colab import drive
#Authorize access to google drive
drive.mount('/content/drive', force_remount=True)



Mounted at /content/drive


In [7]:
#Dictionary to map US states to State Codes
us_state_to_abbrev = {
"Alabama": "AL",
"Alaska": "AK",
"Arizona": "AZ",
"Arkansas": "AR",
"California": "CA",
"Colorado": "CO",
"Connecticut": "CT",
"Delaware": "DE",
"Florida": "FL",
"Georgia": "GA",
"Hawaii": "HI",
"Idaho": "ID",
"Illinois": "IL",
"Indiana": "IN",
"Iowa": "IA",
"Kansas": "KS",
"Kentucky": "KY",
"Louisiana": "LA",
"Maine": "ME",
"Maryland": "MD",
"Massachusetts": "MA",
"Michigan": "MI",
"Minnesota": "MN",
"Mississippi": "MS",
"Missouri": "MO",
"Montana": "MT",
"Nebraska": "NE",
"Nevada": "NV",
"New Hampshire": "NH",
"New Jersey": "NJ",
"New Mexico": "NM",
"New York": "NY",
"North Carolina": "NC",
"North Dakota": "ND",
"Ohio": "OH",
"Oklahoma": "OK",
"Oregon": "OR",
"Pennsylvania": "PA",
"Rhode Island": "RI",
"South Carolina": "SC",
"South Dakota": "SD",
"Tennessee": "TN",
"Texas": "TX",
"Utah": "UT",
"Vermont": "VT",
"Virginia": "VA",
"Washington": "WA",
"West Virginia": "WV",
"Wisconsin": "WI",
"Wyoming": "WY",
"District of Columbia": "DC",
"American Samoa": "AS",
"Guam": "GU",
"Northern Mariana Islands": "MP",
"Puerto Rico": "PR",
"United States Minor Outlying Islands": "UM",
"U.S. Virgin Islands": "VI",
}

# **Fetch irradiance and temperature for given citites**

In [9]:
# Fetch the lat-lon file into a dataframe
dfll = pd.read_excel('/content/drive/My Drive/ColabNotebooks/inputFolder/lat-lon.xlsx')

# Add two new columns "City Name" and "State Code/Country Name"
dfll["City Name"] = pd.NaT
dfll["State Code/Country Name"] = pd.NaT

# Use geolocator to fetch city and state code (for US cities)/ country name using latitude and longitude and populate dfll dataframe
geolocator = Nominatim(user_agent='myapplication')
for index, row in dfll.iterrows():
  address = geolocator.reverse(str(row[0])+","+str(row[1])).raw['address']
  dfll.loc[index,"City Name"]=address.get('city', '')
  if address.get('country', '')== "United States":
    dfll.loc[index,"State Code/Country Name"]= us_state_to_abbrev.get(address.get('state', ''))
  else:
    dfll.loc[index,"State Code/Country Name"]= address.get('country', '')

#Drop NaT values and update the lat-lon file
dfll = dfll.dropna()

In [11]:
#fetch the city-name file into a dataframe
dfcity = pd.read_excel('/content/drive/My Drive/ColabNotebooks/inputFolder/city-name.xlsx')

#Add two new columns "Latitude" and "Longitude"
dfcity["Latitude"] = pd.NaT
dfcity["Longitude"] = pd.NaT

# Use geolocator to latitude and longitude using city and state code (for US cities)/ country name and populate dfcity dataframe
geolocator = Nominatim(user_agent='myapplication')
for index, row in dfcity.iterrows():
  location = geolocator.geocode(row[0]+","+row[1],timeout=1000)
  if not location == None:
    dfcity.loc[index,"Latitude"]=location.latitude
    dfcity.loc[index,"Longitude"]=location.longitude

#Drop NaT values and update the lat-lon file
dfcity=dfcity.dropna()

In [12]:
#Concatenate lat-lon and city-name dataframes without duplicates
df_irradiance_temp = pd.concat([dfcity,dfll], axis = 0).drop_duplicates().reset_index(drop=True)

In [13]:
#Add four new columns "Irradiance", "Avg temp", "Max temp", and "Min temp"
df_irradiance_temp["Irradiance"] = pd.NaT
df_irradiance_temp["Avg temp"]= pd.NaT
df_irradiance_temp["Max temp"]= pd.NaT
df_irradiance_temp["Min temp"]= pd.NaT
for index,row in df_irradiance_temp.iterrows():
  #Get irradiance and temperature for all the latitude and longitude values in the dataframe
  json_data = json.loads(requests.get('https://power.larc.nasa.gov/api/temporal/monthly/point?start=2016&end=2020&latitude='+str(row[2])+'&longitude='+str(row[3])+'&community=re&parameters=ALLSKY_SFC_SW_DWN%2CT2M%2CT2M_MAX%2CT2M_MIN&format=json&header=true').text)
  irradiance = json_data['properties']['parameter']['ALLSKY_SFC_SW_DWN']
  AvgTemp = json_data['properties']['parameter']['T2M']
  maxTemp = json_data['properties']['parameter']['T2M_MAX']
  minTemp = json_data['properties']['parameter']['T2M_MIN']
  df_irradiance_temp.loc[index,"Irradiance"] = (irradiance['201613']+irradiance['201713']+irradiance['201813']+irradiance['201913']+irradiance['202013'])/5
  df_irradiance_temp.loc[index,"Avg temp"] = (AvgTemp['201613']+AvgTemp['201713']+AvgTemp['201813']+AvgTemp['201913']+AvgTemp['202013'])/5
  df_irradiance_temp.loc[index,"Max temp"] = (maxTemp['201613']+maxTemp['201713']+maxTemp['201813']+maxTemp['201913']+maxTemp['202013'])/5
  df_irradiance_temp.loc[index,"Min temp"] = (minTemp['201613']+minTemp['201713']+minTemp['201813']+minTemp['201913']+minTemp['202013'])/5

In [15]:
#Creating a new irradiance-temp.xlsx in the output folder
df_irradiance_temp.to_excel('/content/drive/My Drive/ColabNotebooks/outputFolder/irradiance-temp.xlsx',index=False)

# **Fetch Average NO2 for given cities**

In [17]:
# Get the contents of NASA Timeseries files through URL
!wget https://so2.gsfc.nasa.gov/no2/pix/time_series/OMNO2_Timeseries_AllCities.csv -q
data=pd.read_csv('OMNO2_Timeseries_AllCities.csv',index_col=False, on_bad_lines='skip',skiprows=7)
df_average_NO2 = pd.DataFrame(data)

In [18]:
sum_med = df_average_NO2.groupby('City')['Median'].mean()

with open('OMNO2_Timeseries_AllCities.csv','r') as f_in:
    f = csv.reader(f_in)
    rows=df_average_NO2.groupby(['City','Country','LonCenter','LatCenter']).agg({'Median': ['mean']})
    rows.columns = ['Average NO2']
    rows = rows.reset_index()

for index, row in rows.iterrows():
  #Get latitude and longitude using city and state code (for US cities)/ country name 
  location = geolocator.geocode(row[0]+","+row[1],timeout=1000)
  if not location == None:
    rows.loc[index,"LatCenter"]=location.latitude
    rows.loc[index,"LonCenter"]=location.longitude

nasa_cities = rows.rename(columns={'City': 'City Name','LonCenter':'Longitude','LatCenter':'Latitude'})

City
Aberdeen     -0.214377
Abidjan      -0.276345
AbuDhabi      2.567037
Abuja         1.008219
Accra        -1.260332
               ...    
Yangoon     -13.507840
Yaounde      -0.735572
Yerevan       1.204523
Zhengzhou     6.623809
Zurich        1.627309
Name: Median, Length: 316, dtype: float64


In [19]:
nasa_cities = nasa_cities.apply(pd.to_numeric, errors='ignore')
df_irradiance_temp = df_irradiance_temp.apply(pd.to_numeric, errors='ignore')
df_irradiance_temp_NO2 = df_irradiance_temp
#NO2data = pd.merge(customer_cities,nasa_cities , on=['Longitude','Latitude'])
df_irradiance_temp_NO2["Average NO2"] = pd.NaT

# Check the nearest city based on latitude and longitude values
for index, row in df_irradiance_temp_NO2.iterrows():
  min_distance=float('inf')
  for nindex, nrow in nasa_cities.iterrows():
    if(distance.distance((row['Latitude'],row['Longitude']), (nrow['Latitude'],nrow['Longitude'])).miles < min_distance):
      min_distance = distance.distance((row['Latitude'],row['Longitude']), (nrow['Latitude'],nrow['Longitude'])).miles
      min_index = nindex
  df_irradiance_temp_NO2.loc[index,'Average NO2'] = nasa_cities['Average NO2'].values[min_index]

In [20]:
#Creating a new Average NO2 data in the output folder
df_irradiance_temp_NO2.to_excel('/content/drive/My Drive/ColabNotebooks/outputFolder/average_no2_data.xlsx',index=False)